# Data updating

In [ ]:
# 농넷 전국도매시장 거래정보 데이터 API 

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import json
import datetime
import multiprocessing
import pickle
from tqdm import tqdm
from glob import glob

In [ ]:
#예측일 기준 하루 전 날 도매 거래정보 데이터 추가
today = datetime.datetime.today()
yesterday = (today - datetime.timedelta(1)).strftime('%Y%m%d') 
url = 'https://www.nongnet.or.kr/api/whlslDstrQr.do?sdate=' # sdate = 날짜

response = urllib.request.urlopen(url+yesterday).read()
response = json.loads(response)

In [ ]:
#data chk
response

In [ ]:
# submission 파일 형태로 변환
data=pd.DataFrame()
for i in tqdm(range(1,0,-1)):
    
    today = datetime.datetime.today()
    yesterday = (today - datetime.timedelta(i)).strftime('%Y%m%d') 
    url = 'https://www.nongnet.or.kr/api/whlslDstrQr.do?sdate=' # sdate = 날짜

    response = urllib.request.urlopen(url+yesterday).read()
    response = json.loads(response)
    if not response['data']:
        response['data']=[{'PUM_NM': 0,
       'LV_NM': 0,
       'TOT_AMT': 0,
       'SAN_NM': 0,
       'SALEDATE': yesterday ,
       'CMP_NM': 0,
       'DAN_NM': 0,
       'WHSAL_NM':0,
       'SIZE_NM': '.',
       'COST': 0,
       'POJ_NM': '상자',
       'TOT_QTY': 0,
       'QTY': 0,
       'KIND_NM': 0,
       'DANQ': 0}]


    data = data.append(response['data'])

In [ ]:
#data chk
data['SALEDATE'].values

In [ ]:
unique_pum = [
        '배추', '무', '양파', '건고추','마늘',
        '대파', '얼갈이배추', '양배추', '깻잎',
        '시금치', '미나리', '당근',
        '파프리카', '새송이', '팽이버섯', '토마토',
    ]
unique_kind = [
        '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
    ]

kind_pum=unique_kind + unique_pum


In [ ]:
df_pum=pd.DataFrame()
for pum in unique_pum:
    df_pum=df_pum.append(data[data['PUM_NM']==pum])

for pum in unique_kind:
    df_pum=df_pum.append(data[data['KIND_NM']==pum])

In [ ]:
df_pum=df_pum.reset_index(drop=True)

In [ ]:
train_dict = {
    'date':[]
}

for sub in unique_pum:
    train_dict[f'{sub}_거래량(kg)'] = []
    train_dict[f'{sub}_가격(원/kg)'] = []

for sub in unique_kind:
    train_dict[f'{sub}_거래량(kg)'] = []
    train_dict[f'{sub}_가격(원/kg)'] = []


days = sorted(data['SALEDATE'].unique())

for day in tqdm(days):
    train_dict['date'].append(day)
    for sub in unique_pum:
        # 날짜별, 품목별, 거래량이 0 이상인 행만 선택
        c = data[(data['SALEDATE']==day) & (data['PUM_NM']==sub) & (data['TOT_QTY']>0)]
        if c.shape[0] == 0:
            train_dict[f'{sub}_거래량(kg)'].append(0)
            train_dict[f'{sub}_가격(원/kg)'].append(0)
        else:
            tot_amt = c['TOT_AMT'].sum().astype(float)
            tot_qty = c['TOT_QTY'].sum().astype(float)
            mean_price = tot_amt/(tot_qty+1e-20)
            train_dict[f'{sub}_거래량(kg)'].append(tot_qty)
            train_dict[f'{sub}_가격(원/kg)'].append(mean_price)

    for sub in unique_kind:
        # 날짜별, 품종별, 거래량이 0 이상인 행만 선택
        c = data[(data['SALEDATE']==day) & (data['KIND_NM']==sub) & (data['TOT_QTY']>0)]
        if c.shape[0] == 0:
            train_dict[f'{sub}_거래량(kg)'].append(0)
            train_dict[f'{sub}_가격(원/kg)'].append(0)
        else:
            tot_amt = c['TOT_AMT'].sum().astype(float)
            tot_qty = c['TOT_QTY'].sum().astype(float)
            mean_price = round(tot_amt/(tot_qty+1e-20))
            tot_qty = round(tot_qty, 1)
            train_dict[f'{sub}_거래량(kg)'].append(tot_qty)
            train_dict[f'{sub}_가격(원/kg)'].append(mean_price)

In [ ]:
df_private=pd.DataFrame(train_dict)

In [ ]:
df_private['date'] = df_private.date.astype(str).str.replace('-','')
df_private['date'] = pd.to_datetime(df_private.date, format='%Y%m%d')

In [ ]:
# 예측일 기준 하루 전 날 21개 품목 거래 가격 데이터 chk
df_private

In [ ]:
# For modifying train, test dataset
df_private.to_csv("new_common.csv",index=False, encoding='UTF-8')

# Updating train & test dataset

In [ ]:
datum5 = pd.read_csv("C:/Users/KISLAB03/Desktop/2021-2/1234/data/train_update.csv")
datum5

In [ ]:
datum6 = pd.read_csv("C:/Users/KISLAB03/Desktop/2021-2/1234/data/test_update.csv")
datum6

In [ ]:
new1003 = pd.read_csv("C:/Users/KISLAB03/Desktop/데이콘 영우형꺼/new_common.csv")
new1003

In [ ]:
# test dataset update
# test dataset에 예측일 하루 전 날 거래 가격 데이터 추가
new_1003 =pd.concat([datum6,new1003], axis=0)
new_1003

In [ ]:
# test data는 29일치로 고정
# 예측일 하루 전 날 데이터가 추가된 대신 test set 제일 첫 날 데이터를 제외한 test dataset 생성
new_new_1003 = new_1003.iloc[1:,:]
new_new_1003

In [ ]:
new_new_1003.to_csv("C:/Users/KISLAB03/Desktop/2021-2/1234/data/test_update.csv", encoding="UTF-8", index=False)

In [ ]:
# test set 제일 첫 날 데이터를 train data로 이동하기 위해 별도 변수화
con_1003 = new_1003.iloc[:1,:]
con_1003

In [ ]:
# train dataset update
train_update= pd.concat([datum5,con_1003], axis=0)

In [ ]:
train_update

In [ ]:
train_update.to_csv("C:/Users/KISLAB03/Desktop/2021-2/1234/data/train_update.csv",index=False)

# 학습 & 추론

In [ ]:
import lightgbm
import warnings
from xgboost import XGBRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# 경고 끄기
pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings(action='ignore')

### Add lag features & Preprocessing

In [ ]:
def preprocessing(temp_df, pum, len_lag) :
    # p_lag, q_lag 추가
    for lag in range(1,len_lag+1) :
      temp_df[f'p_lag_{lag}'] = -1
      temp_df[f'q_lag_{lag}'] = -1
      for index in range(lag, len(temp_df)) :
        temp_df.loc[index, f'p_lag_{lag}'] = temp_df[f'{pum}_가격(원/kg)'][index-lag] #1일전, 2일전, ... 가격을 feature로 추가
        temp_df.loc[index, f'q_lag_{lag}'] = temp_df[f'{pum}_거래량(kg)'][index-lag] #1일전, 2일전, ... 거래량을 feature로 추가

    # month 추가
    temp_df['date'] = pd.to_datetime(temp_df['date'])
    temp_df['month'] = temp_df['date'].dt.month

    # 예측 대상(1w,2w,4w) 추가
    for week in ['1_week','2_week','4_week'] :
      temp_df[week] = 0
      n_week = int(week[0])
      for index in range(len(temp_df)) :
        try : temp_df[week][index] = temp_df[f'{pum}_가격(원/kg)'][index+7*n_week]
        except : continue

    # 불필요한 column 제거        
    temp_df = temp_df.drop(['date',f'{pum}_거래량(kg)',f'{pum}_가격(원/kg)'], axis=1)
    
    return temp_df

### Data load & Scaling

In [ ]:
train = pd.read_csv('C:/Users/KISLAB03/Desktop/2021-2/1234/data/train_update.csv')
train

## 학습

In [ ]:
def nmae(week_answer, week_submission):
    answer = week_answer
    target_idx = np.where(answer!=0)
    true = answer[target_idx]
    pred = week_submission[target_idx]
    score = np.mean(np.abs(true-pred)/true)
    
    return score


def at_nmae(pred, y_true):
    #y_true = dataset.get_label()
    week_1_answer = y_true[0::3]
    week_2_answer = y_true[1::3]
    week_4_answer = y_true[2::3]
    
    week_1_submission = pred[0::3]
    week_2_submission = pred[1::3]
    week_4_submission = pred[2::3]
    
    score1 = nmae(week_1_answer, week_1_submission)
    score2 = nmae(week_2_answer, week_2_submission)
    score4 = nmae(week_4_answer, week_4_submission)
    
    score = (score1+score2+score4)/3
    
    return 'score', score, False

In [ ]:
unique_pum = [
    '배추', '무', '양파', '건고추','마늘',
    '대파', '얼갈이배추', '양배추', '깻잎',
    '시금치', '미나리', '당근',
    '파프리카', '새송이', '팽이버섯', '토마토',
]

unique_kind = [
    '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
]

### Ensemble(XGBoost, AdaBoost, LGBM, RandomForest)

In [ ]:
model_dict = {}
model_dict2 = {}
model_dict3 = {}
model_dict4 = {}
split = 28 #validation

for pum in tqdm(unique_pum + unique_kind):
    # 품목 품종별 전처리
    temp_df = train[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
    temp_df = preprocessing(temp_df, pum, len_lag=28)
    
    # 주차별(1,2,4w) 학습
    for week_num in [1,2,4] :
        x = temp_df[temp_df[f'{week_num}_week']>0].iloc[:,:-3]
        y = temp_df[temp_df[f'{week_num}_week']>0][f'{week_num}_week']
        
        #train, test split
        x_train = x[:-split]
        y_train = y[:-split]
        x_valid = x[-split:]
        y_valid = y[-split:]

# 모델학습 및 그리드서치

        model1 = XGBRegressor()
        p = {'max_depth' : [3,4,5,6] , 'n_estimators': [12,24,32], 'learning_rate':[0.01, 0.1], 'gamma': [0.5, 1, 2], 'random_state':[2021]}
        gcv = GridSearchCV(model1, param_grid=p, cv=5,  n_jobs=-1)
        gcv.fit(x_train, y_train)
        model2 = XGBRegressor(max_depth=gcv.best_params_['max_depth'], learning_rate=gcv.best_params_['learning_rate'],
                             n_estimators = gcv.best_params_['n_estimators'], gamma=gcv.best_params_['gamma'])
        

        
        model3 = AdaBoostRegressor()
        p2 = {
            'learning_rate': [0.1, 0.03],
            'n_estimators': [50, 100, 200]
        }
        gcv2 = GridSearchCV(model3, param_grid=p2, cv=5,  n_jobs=-1)
        gcv2.fit(x_train,y_train)
        model4 = AdaBoostRegressor(n_estimators=gcv2.best_params_['n_estimators'], learning_rate=gcv2.best_params_['learning_rate'])
        
        
        model5 = LGBMRegressor()
        p3 = {
            'num_leaves': [7, 14, 21],
            'learning_rate': [0.1, 0.03],
            'max_depth': [3, 5, 10],
            'n_estimators': [50, 100, 200]
        }
        gcv3 = GridSearchCV(model5, param_grid=p3, cv=5,  n_jobs=-1)
        gcv3.fit(x_train,y_train)
        model6 = LGBMRegressor(learning_rate=gcv3.best_params_['learning_rate'],n_estimators=gcv3.best_params_['n_estimators'],
                              num_leaves = gcv3.best_params_['num_leaves'], max_depth=gcv3.best_params_['max_depth'])
        
        
        model7 = RandomForestRegressor()
        p4 = {'n_estimators': [50,60,70],
            'max_depth': [10,15,20],
            'max_leaf_nodes':[50,100,200],
            'max_features':['auto','sqrt','log2']}
        gcv4 = GridSearchCV(model7, param_grid=p4, cv=5,  n_jobs=-1)
        gcv4.fit(x_train,y_train)
        model8 = RandomForestRegressor(n_estimators=gcv4.best_params_['n_estimators'],max_depth=gcv4.best_params_['max_depth'],
                                     max_leaf_nodes=gcv4.best_params_['max_leaf_nodes'],max_features=gcv4.best_params_['max_features'] )
        
        
        model_dict[f'{pum}_model_{week_num}'] = model2.fit(x_train, y_train)
        model_dict2[f'{pum}_model_{week_num}'] = model4.fit(x_train, y_train)
        model_dict3[f'{pum}_model_{week_num}'] = model6.fit(x_train, y_train)
        model_dict4[f'{pum}_model_{week_num}'] = model8.fit(x_train, y_train)
        
        pred1 = model_dict[f'{pum}_model_{week_num}'].predict(x_valid)
        pred2 = model_dict2[f'{pum}_model_{week_num}'].predict(x_valid)
        pred3 = model_dict3[f'{pum}_model_{week_num}'].predict(x_valid)
        pred4 = model_dict4[f'{pum}_model_{week_num}'].predict(x_valid)

In [ ]:
#학습이 잘 되었는지 chk
pred2.shape

## 추론

In [ ]:
submission = pd.read_csv("C:/Users/KISLAB03/Desktop/2021-2/1234/sample_submission.csv")
public_date_list = submission[submission['예측대상일자'].str.contains('2021')]['예측대상일자'].str.split('+').str[0].unique()
# ['2021-09-28', ...]

date = public_date_list[37] # 10/08기준 [10]에서 1씩 늘려가기 ex.10/09 > [11], 10월은 해당 날짜 +2

test = pd.read_csv('C:/Users/KISLAB03/Desktop/2021-2/1234/data/test_update.csv')

for pum in unique_pum + unique_kind:
# 예측기준일에 대해 전처리
    temp_test = pd.DataFrame([{'date' : date}]) #예측기준일
    alldata = pd.concat([test, temp_test], sort=False).reset_index(drop=True)
    alldata = alldata[['date', f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']].fillna(0)
    alldata = alldata.iloc[-29:].reset_index(drop=True)
    alldata = preprocessing(alldata, pum, len_lag=28)
    temp_test = alldata.iloc[-1].astype(float)
# 개별 모델을 활용하여 1,2,4주 후 가격 예측
    for week_num in [1,2,4] :
        temp_model = model_dict[f'{pum}_model_{week_num}']
        result = temp_model.predict(temp_test[:-3].values.reshape(1, -1))
        condition = (submission['예측대상일자']==f'{date}+{week_num}week')
        idx = submission[condition].index
        submission.loc[idx, f'{pum}_가격(원/kg)'] = result[0]

In [ ]:
submission.to_csv('C:/Users/KISLAB03/Desktop/2021-2/1234/xgb_update.csv',index=False)

In [ ]:
submission = pd.read_csv('C:/Users/KISLAB03/Desktop/2021-2/1234/sample_submission.csv')
public_date_list = submission[submission['예측대상일자'].str.contains('2021')]['예측대상일자'].str.split('+').str[0].unique()
# ['2021-09-28', ...]

date = public_date_list[37] # 10/08기준 [10]에서 1씩 늘려가기 ex.10/09 > [11], 10월은 해당 날짜 +2

test = pd.read_csv('C:/Users/KISLAB03/Desktop/2021-2/1234/data/test_update.csv')
for pum in unique_pum + unique_kind:
# 예측기준일에 대해 전처리
    temp_test = pd.DataFrame([{'date' : date}]) #예측기준일
    alldata = pd.concat([test, temp_test], sort=False).reset_index(drop=True)
    alldata = alldata[['date', f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']].fillna(0)
    alldata = alldata.iloc[-29:].reset_index(drop=True)
    alldata = preprocessing(alldata, pum, len_lag=28)
    temp_test = alldata.iloc[-1].astype(float)
# 개별 모델을 활용하여 1,2,4주 후 가격 예측
    for week_num in [1,2,4] :
        temp_model = model_dict2[f'{pum}_model_{week_num}']
        result = temp_model.predict(temp_test[:-3].values.reshape(1, -1))
        condition = (submission['예측대상일자']==f'{date}+{week_num}week')
        idx = submission[condition].index
        submission.loc[idx, f'{pum}_가격(원/kg)'] = result[0]

In [ ]:
submission.to_csv('C:/Users/KISLAB03/Desktop/2021-2/1234/Ada_update.csv',index=False)

In [ ]:
submission = pd.read_csv('C:/Users/KISLAB03/Desktop/2021-2/1234/sample_submission.csv')
public_date_list = submission[submission['예측대상일자'].str.contains('2021')]['예측대상일자'].str.split('+').str[0].unique()
# ['2021-09-28', ...]

date = public_date_list[37] # 10/08기준 [10]에서 1씩 늘려가기 ex.10/09 > [11], 10월은 해당 날짜 +2

test = pd.read_csv('C:/Users/KISLAB03/Desktop/2021-2/1234/data/test_update.csv')
for pum in unique_pum + unique_kind:
# 예측기준일에 대해 전처리
    temp_test = pd.DataFrame([{'date' : date}]) #예측기준일
    alldata = pd.concat([test, temp_test], sort=False).reset_index(drop=True)
    alldata = alldata[['date', f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']].fillna(0)
    alldata = alldata.iloc[-29:].reset_index(drop=True)
    alldata = preprocessing(alldata, pum, len_lag=28)
    temp_test = alldata.iloc[-1].astype(float)
# 개별 모델을 활용하여 1,2,4주 후 가격 예측
    for week_num in [1,2,4] :
        temp_model = model_dict3[f'{pum}_model_{week_num}']
        result = temp_model.predict(temp_test[:-3].values.reshape(1, -1))
        condition = (submission['예측대상일자']==f'{date}+{week_num}week')
        idx = submission[condition].index
        submission.loc[idx, f'{pum}_가격(원/kg)'] = result[0]

In [ ]:
submission.to_csv('C:/Users/KISLAB03/Desktop/2021-2/1234/LGBM_update.csv',index=False)

In [ ]:
submission = pd.read_csv('C:/Users/KISLAB03/Desktop/2021-2/1234/sample_submission.csv')
public_date_list = submission[submission['예측대상일자'].str.contains('2021')]['예측대상일자'].str.split('+').str[0].unique()
# ['2021-09-28', ...]

date = public_date_list[37] # 10/08기준 [10]에서 1씩 늘려가기 ex.10/09 > [11], 10월은 해당 날짜 +2

test = pd.read_csv('C:/Users/KISLAB03/Desktop/2021-2/1234/data/test_update.csv')
for pum in unique_pum + unique_kind:
# 예측기준일에 대해 전처리
    temp_test = pd.DataFrame([{'date' : date}]) #예측기준일
    alldata = pd.concat([test, temp_test], sort=False).reset_index(drop=True)
    alldata = alldata[['date', f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']].fillna(0)
    alldata = alldata.iloc[-29:].reset_index(drop=True)
    alldata = preprocessing(alldata, pum, len_lag=28)
    temp_test = alldata.iloc[-1].astype(float)
# 개별 모델을 활용하여 1,2,4주 후 가격 예측
    for week_num in [1,2,4] :
        temp_model = model_dict4[f'{pum}_model_{week_num}']
        result = temp_model.predict(temp_test[:-3].values.reshape(1, -1))
        condition = (submission['예측대상일자']==f'{date}+{week_num}week')
        idx = submission[condition].index
        submission.loc[idx, f'{pum}_가격(원/kg)'] = result[0]

In [ ]:
submission.to_csv('C:/Users/KISLAB03/Desktop/2021-2/1234/RF_update.csv',index=False)

### Weight(가중치)

In [ ]:
Xgb = pd.read_csv('C:/Users/KISLAB03/Desktop/2021-2/1234/xgb_update.csv')
Ada = pd.read_csv('C:/Users/KISLAB03/Desktop/2021-2/1234/Ada_update.csv')
Lgbm = pd.read_csv('C:/Users/KISLAB03/Desktop/2021-2/1234/LGBM_update.csv')
Rf = pd.read_csv('C:/Users/KISLAB03/Desktop/2021-2/1234/RF_update.csv')

Rf.iloc[225:228,1:]

In [ ]:
# 10/08기준 [144:147, 1:]에서 3씩 늘려가기 ex.10/09 > [147:150, 1:]
con_all= Xgb.iloc[225:228,1:]*0.1 + Ada.iloc[225:228,1:]*0.35 + Lgbm.iloc[225:228,1:]*0.45 + Rf.iloc[225:228,1:]*0.1

## Submission

In [ ]:
conjun = pd.concat([Xgb['예측대상일자'],con_all],axis=1)
conjun=conjun.fillna(0)

# conjun.iloc[225:228,]

conjun.to_csv("C:/Users/KISLAB03/Desktop/2021-2/1234/finally_update.csv", index=False, encoding='UTF-8')